<a href="https://colab.research.google.com/github/BecomeAllan/S2Search/blob/main/SemanticScholarSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consumindo a API do SemanticScholar

A seguir, tem uma classe sem parámetros chamada `Search()`, que ao instanciar-la em uma variável é possível fazer pesquisas sobre papers utilizando a api do SemanticScholar, dentre os parâmetros temos:

- Buscar: Pesquisas sobre tópicos onde adicionar tópicos utiliza-se + (mais) e remover tópicos usamos - (menos)

- Fields: O que será retornado como dados. Para utilizar, escolha dentre as opções sem utilizar espaço e separadas de virgulas:
  - externalIds
  - url
  - title
  - abstract
  - venue 
  - year 
  - referenceCount
  - citationCount
  - influentialCitationCount
  - isOpenAccess
  - fieldsOfStudy
  - authors 

- Offset: Número que começa a puxar a partir da ordem dele a lista de papers. (0 seria o primeiro)

- Limite: Número de papers a ser retornados (Máx. 10.000)

**Obs:** A api do SemanticScholar disponibiliza 100 query's a cada 5 min, no qual apenas retorna no máx. 100 resutados (limite). Assim a cada 5 min, é possível puxar 10.000 papers.

In [71]:
#@title Classe para pesquisa no SemanticScholar
import IPython
from google.colab import output
import pandas as pd

class Search():
  def __init__(self):
    self.data = ""

    self._start()
  
  def _start(self):

    output.register_callback('notebook.searching', self._searching)
    output.register_callback('notebook.AddListItem', self._add_list_item)
    output.register_callback('notebook.mergeData', self._merge_data)
    # output.register_callback('notebook.error', self._error)

    display(IPython.display.HTML('''
        <label for="query">Buscar: </label>
        <input type="text" id="query" value="Machine Learning+Deep Learning" style="width: 400px;"/>
        <br/>
        <br/>
        
        <label for="fields">Fields: </label>
        <input type="text" id="fields" value="title,abstract,isOpenAccess,fieldsOfStudy" style="width: 400px;"/>
        <br/>
        <br/>
 
        <label for="limit">Limite: </label>
        <input type="text" id="limit" value="10" style="width: 50px;"/><br/>
        <br/>

        <label for="limit">Offset: </label>
        <input type="text" id="offset" value="0" style="width: 50px;"/><br/>
        <br/>

        <button id='button'>Pesquisar</button>
        <br/>
        <br/>
        <script>
          document.querySelector('#button').onclick = async () => {

            var search = document.getElementById("query").value
            var fields = document.getElementById("fields").value
            var limit = document.getElementById("limit").value
            var offset = document.getElementById("offset").value

            google.colab.kernel.invokeFunction('notebook.searching', [], {});

            if (limit >100) {
              var number = limit
              var data = ""
              var promises = []
              
              for (let index = 0; index < Math.floor(limit/100); index++) {
                offset = 100*(index) + offset

                promises.push(
                  fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offset}&limit=100&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
                )
              }
              
              if (limit%100 !== 0) { 
              var rest = limit%100
              offset = offset+100

              promises.push(
              fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offset}&limit=${rest}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
              )}

              await Promise.all(promises).then(data=>{
                google.colab.kernel.invokeFunction('notebook.mergeData', [data], {})
              })

            } else {
            await fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offset}&limit=${limit}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {
      return(google.colab.kernel.invokeFunction('notebook.AddListItem', [res], {}))})
            }
      
     // .catch(err=>{
    //   google.colab.kernel.invokeFunction('notebook._error', [], {});
    //   })


          };
        </script>
        '''))
    

  def _error(self):
    print("Deu ruim")

  def _searching(self):
    with output.use_tags('some_outputs'):
      print("\n\nPesquisando...")
      sys.stdout.flush();

  def _merge_data(self, data):
    if self.data == "":
      output.clear(output_tags='some_outputs')
      print(f"Achou {data[0]['total']} papers.")

      self.data = pd.concat([pd.DataFrame(x['data']) for x in data], ignore_index=True )   
    else:
      self.data = pd.concat([self.data, data], ignore_index=True)

    print(self.data.head())

  def merge(self, data):
    self.data = pd.concat([self.data, data], ignore_index=True ) 

  def _add_list_item(self, value):
    output.clear(output_tags='some_outputs')
    print(f"Achou {value['total']} papers.")

    if self.data == "":
      self.data = pd.DataFrame(value['data'])
    
    print(self.data.head())



In [72]:
Resultados = Search()

Achou 648799 papers.
                                    paperId  ...       fieldsOfStudy
0  7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f  ...  [Computer Science]
1  59444b096f7c8a561d540102e8b5bfb189edabc6  ...                None
2  eee313380ccb45807ea0afa3c1df86f6b48b8867  ...  [Computer Science]

[3 rows x 5 columns]


In [73]:
# Os dados ficam na variável data
Resultados.data

,paperId,title,abstract,isOpenAccess,fieldsOfStudy
0,7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f,Rainfall Prediction using Machine Learning & D...,"In India, Agriculture is the key point for sur...",False,[Computer Science]
1,59444b096f7c8a561d540102e8b5bfb189edabc6,Perspectives on the Impact of Machine Learning...,The fields of machining learning and artificia...,True,None
2,eee313380ccb45807ea0afa3c1df86f6b48b8867,Model Selection and Evaluation for Machine Lea...,Materials processing is a critical subset of m...,False,[Computer Science]


In [74]:
Resultados_2 = Search()

Achou 648798 papers.
                                    paperId  ...       fieldsOfStudy
0  846ff7afb7670d62f88b4a8cc99d306ffb81b075  ...          [Medicine]
1  5dc53e50148b01fe8b9536eb79fa6b1dce924174  ...          [Medicine]
2  7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f  ...  [Computer Science]

[3 rows x 5 columns]


In [75]:
Resultados_2.data

,paperId,title,abstract,isOpenAccess,fieldsOfStudy
0,846ff7afb7670d62f88b4a8cc99d306ffb81b075,"Artificial Intelligence, Machine Learning, Dee...",This article was presented at the 2017 annual ...,False,[Medicine]
1,5dc53e50148b01fe8b9536eb79fa6b1dce924174,Promising Artificial Intelligence‐Machine Lear...,Abstract: The lifestyle of modern society has ...,False,[Medicine]
2,7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f,Rainfall Prediction using Machine Learning & D...,"In India, Agriculture is the key point for sur...",False,[Computer Science]


In [76]:
Resultados_2.merge(Resultados.data)

Resultados_2.data.drop_duplicates("paperId")

,paperId,title,abstract,isOpenAccess,fieldsOfStudy
0,846ff7afb7670d62f88b4a8cc99d306ffb81b075,"Artificial Intelligence, Machine Learning, Dee...",This article was presented at the 2017 annual ...,False,[Medicine]
1,5dc53e50148b01fe8b9536eb79fa6b1dce924174,Promising Artificial Intelligence‐Machine Lear...,Abstract: The lifestyle of modern society has ...,False,[Medicine]
2,7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f,Rainfall Prediction using Machine Learning & D...,"In India, Agriculture is the key point for sur...",False,[Computer Science]
4,59444b096f7c8a561d540102e8b5bfb189edabc6,Perspectives on the Impact of Machine Learning...,The fields of machining learning and artificia...,True,None
5,eee313380ccb45807ea0afa3c1df86f6b48b8867,Model Selection and Evaluation for Machine Lea...,Materials processing is a critical subset of m...,False,[Computer Science]
